# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

In [10]:
dataframe = patents.select("PATENT", "POSTATE")

In [11]:
dataframe.show(5)

+-------+-------+
| PATENT|POSTATE|
+-------+-------+
|3070801|   null|
|3070802|     TX|
|3070803|     IL|
|3070804|     OH|
|3070805|     CA|
+-------+-------+
only showing top 5 rows



In [12]:
dataframe.cache()

DataFrame[PATENT: int, POSTATE: string]

In [44]:
dataframe1 = citations.join(dataframe, citations.CITING==dataframe.PATENT, "left").drop("PATENT").withColumnRenamed("POSTATE", "CITING_POSTATE")

In [45]:
dataframe1.show(10)

+-------+-------+--------------+
| CITING|  CITED|CITING_POSTATE|
+-------+-------+--------------+
|3858258|1331793|            CA|
|3858258|1540798|            CA|
|3858527| 924225|          null|
|3858527|2444326|          null|
|3858527|2705120|          null|
|3858527|2967080|          null|
|3858527|3602157|          null|
|3858527|3638586|          null|
|3858527|3699902|          null|
|3858560| 957631|            IN|
+-------+-------+--------------+
only showing top 10 rows



Joining citations with patents table on 'citing'. 

In [47]:
dataframe2 = dataframe1.join(dataframe, dataframe1.CITED==dataframe.PATENT, "left").drop("PATENT").withColumnRenamed("POSTATE", "CITED_POSTATE")

In [48]:
dataframe2.show(10)

+-------+-----+--------------+-------------+
| CITING|CITED|CITING_POSTATE|CITED_POSTATE|
+-------+-----+--------------+-------------+
|4305315| 2366|            MN|         null|
|4192521| 2366|          null|         null|
|4253355| 2366|            MN|         null|
|5580635| 5156|            WI|         null|
|4976561| 5518|          null|         null|
|4480374| 5803|            MN|         null|
|5123817| 6620|          null|         null|
|4115020| 7240|          null|         null|
|4727698| 7253|            CA|         null|
|4360982| 7340|            IA|         null|
+-------+-----+--------------+-------------+
only showing top 10 rows



Re-joining the dataframe with patent state on 'cited'.

In [50]:
dataframe3 = dataframe2.filter((dataframe2.CITING_POSTATE != "null") & (dataframe2.CITED_POSTATE != "null"))

In [51]:
dataframe3.show(10)

+-------+-------+--------------+-------------+
| CITING|  CITED|CITING_POSTATE|CITED_POSTATE|
+-------+-------+--------------+-------------+
|4496943|3071753|            NJ|           MN|
|4345315|3071753|            TN|           MN|
|4120573|3071753|            IL|           MN|
|3949375|3071753|            NJ|           MN|
|4271479|3071753|            NY|           MN|
|4280448|3072100|            KS|           IL|
|3861359|3072100|            IL|           IL|
|4138968|3072100|            KS|           IL|
|3894516|3072100|            OK|           IL|
|4572109|3072100|            SC|           IL|
+-------+-------+--------------+-------------+
only showing top 10 rows



Filtering out the rows where state has null values.

In [53]:
dataframe4 = dataframe3.filter("CITING_POSTATE == CITED_POSTATE")

In [55]:
dataframe4.show(10)

+-------+-------+--------------+-------------+
| CITING|  CITED|CITING_POSTATE|CITED_POSTATE|
+-------+-------+--------------+-------------+
|4383847|3177062|            AL|           AL|
|3878730|3610053|            AL|           AL|
|3877317|3610053|            AL|           AL|
|4601891|3733191|            AL|           AL|
|4237106|3733191|            AL|           AL|
|4028087|3733191|            AL|           AL|
|4427432|3969483|            AL|           AL|
|4246248|3969483|            AL|           AL|
|4090893|3974004|            AL|           AL|
|4429634|3974004|            AL|           AL|
+-------+-------+--------------+-------------+
only showing top 10 rows



Filtering the dataframe such that citing_state = cited_state to get co-citations counts.

In [56]:
dataframe5 = dataframe4.drop("CITING_POSTATE", "CITED", "CITED_POSTATE").groupby("CITING").count()

In [57]:
dataframe5.show(10)

+-------+-----+
| CITING|count|
+-------+-----+
|4214280|    2|
|4712293|    5|
|5968543|    8|
|5723969|    4|
|5279657|    2|
|5489298|   16|
|5354420|    1|
|5976783|    5|
|4899423|    1|
|5550181|    5|
+-------+-----+
only showing top 10 rows



Performing groupBy based on 'citing' to get the co-citations count.

In [58]:
dataframe6 = patents.join(dataframe5, patents.PATENT == dataframe5.CITING, "left").drop("CITING")

In [59]:
dataframe6.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|3070853| 1963| 1096|   null|     US|     FL|    null|      1|  null|    49|  5|    59| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null| null|
|3071083| 1963| 1096|   null|     AT|   null|    null|      1|  null|   104|  5|    55| null|       3|    null|    0.0|    null|    null|    null|    null|    null|    null|    null| null|
|3071452| 1963| 1096|   null|     BE|   null|    null| 

Joining the co-citations count with patent table.

In [60]:
dataframe7 = dataframe6.na.fill({'count':0})

In [61]:
dataframe8 = dataframe7.orderBy('count', ascending = False)

In [62]:
dataframe8.show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  null|   326|  4|    46|  159|       0|     1.0|   null|  0.6186|    null|  4.8868|  0.0455|   0.044|    null|    null|  125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  null|   114|  5|    55|  200|       0|   0.995|   null|  0.7201|    null|   12.45|     0.0|     0.0|    null|    null|  103|
|6008204| 1999|14606|   1998|     US|     CA|  749584| 

Sorting the entire dataframe in descending order of co-citations count.